## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Импортируем библиотеку pandas, импортируем модуль pymystem3 и Counter из collections и прочитаем файл с исходными данными. Сохраним файл в переменной df.

In [1]:
import pandas as pd
#!pip install pymystem3
from pymystem3 import Mystem
from collections import Counter
df = pd.read_csv('data.csv')

Получим общую информацию о данных.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Мы обнаружили, что пропущено одинаковое количество значения в столбцах days_employed и total_income.
Рассмотрим все столбцы и их типы данных:
* children — количество детей в семье - int;

* days_employed — общий трудовой стаж в днях - float;

* dob_years — возраст клиента в годах - int;

* education — уровень образования клиента - object;

* education_id — идентификатор уровня образования - int;

* family_status — семейное положение - object;

* family_status_id — идентификатор семейного положения - int;

* gender — пол клиента - object;

* income_type — тип занятости - object;

* debt — имел ли задолженность по возврату кредитов - int;

* total_income — ежемесячный доход - float;

* purpose — цель получения кредита - object.



Выведем первые десять строк таблицы.

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце days_employed мы видим некорректные данные - есть отрицательные значений, а также значения с очень большим количеством дней, это может быть связано с тем, что эти данные загодированы по какой-то системе или с тем? что они ошибочные, данные из этого столбца будут проигнорированы и не будут принимать участие в анализе - для ответа на поставленные вопросы эти данные не потребуются. Также в столбце education есть дубликаты данных - одинаковые данные внесены в разных регистрах. 

Предположим, что пропущенные данные в столбце total_income и days_employed связаны с тем, что клиент является безработным и не имеет дохода и стажа работы. Выведем на экран список уникальных типов занятости для клиентов без дохода total_income.

In [4]:
df['income_type'][df['total_income'].isnull()].unique()

array(['пенсионер', 'госслужащий', 'компаньон', 'сотрудник',
       'предприниматель'], dtype=object)

Как оказалось отстутвие данных по доходам как раз не у безработных людей, а у людей с доходом. Наша гипотеза не подтвердилась, это означает, что нам предстоит заполнить пропуски в этом столбце.

Сосчитаем строки в которых одновременно пропущены значения и в столбце days_employed и total_income. 

In [5]:
df[(df['days_employed'].isnull()) & (df['total_income'].isnull())].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

2174 строки в которых одновременно пропущены значения в обоих столбцах. Это означает, что данные пропущены у одних и тех же строк. Скорее всего отсутсвие этих данных говорит о том, что эти данные не были получены от клиента или у банка нет доступа к расчету поступлений на расчетнй счет клиента (например доходы клиент получает на счет в другой банк).

### Вывод

Исходные данные содержат информацию о клиентах банка и об их кредитной истории (имел ли клиент задолженность по возврату кредитов или нет).
В таблице содержится 12 столбцов и 21525 строк. Пропущено 2174 значения одновременно по двум столбцам days_employed и total_income.
Нам предстоит заполнить пропущенные значения и найти дубликаты. А так же предстоит найти ошибки в данных.

### Шаг 2. Предобработка данных

### Обработка пропусков

Заполним пропуски в столбце total_income с группировкой по нескольким переменным.

Создадим переменную medians в которой сохраним таблицу сгрупированную по столбцам gedner, education_id, income_type и применим метод agg(), который будет рассчитывать медианное значение по ключу total_income.

In [6]:
print('Количество пропусков в total_income до замены:', df['total_income'].isna().sum())

Количество пропусков в total_income до замены: 2174


In [7]:
medians = (df.groupby(['gender', 'education_id', 'income_type']).agg({'total_income': 'median'}).rename(columns = {'total_income':'total_income_median'}))
df = df.merge(medians, on = ['gender', 'education_id', 'income_type'], how = 'left')
df.loc[df['total_income'].isna(), 'total_income'] = df['total_income_median']

In [8]:
print('Количество пропусков в total_income после замены:', df['total_income'].isna().sum())

Количество пропусков в total_income после замены: 1


Проверим в какой строке осталось пустое значение, выведем её на экран.

In [9]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_median
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости,NaN


Мы группировали наши данные по трем метрикам - образование, пол и вид деятельности. Проверим сколько в наших данных есть предпринимателей, мужчин с высшим образованием.

In [10]:
df[(df['education'] == 'высшее') & (df['income_type'] == 'предприниматель') & (df['gender'] == 'M')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_median
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости,NaN


По таким критерия подходит только одна строка. Удалим эту строку.

In [11]:
df.dropna(subset = ['total_income'], inplace=True)

Удалим столбец с расчитанной медианой и выведем общую информацию о таблице.

In [12]:
df = df.drop(['total_income_median'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  float64
 11  purpose           21524 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Все пропуски в столбце о доходе заполнены.

Проверим не осталось ли ошибок в наших данных. Наше задание выяснить влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.
Проверим, какие у нас есть клиенты по количеству детей в семье.

In [13]:
df['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Предположим, что в данных ошибка по причине неверного внесения информации - кто-то любит стучать по клавиатуре. Мы видим отрицательное значение у 47 клиентов, а у 76 клиентов аж по 20 детей. Странно, т.к. количество клиентов у кого больше 4-х детей всего лишь 50. Заменим отрицательные значения на положительные и заменим число 20 на 2 (т.к. скорее всего это опечатка).

In [14]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)
df['children'].value_counts()

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Теперь больше похоже на правду.

Посмотрим на нашу таблицу методом describe - не осталось ли у нас ошибочных значений?

In [15]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21524.000000,19351.000000,21524.000000,21524.000000,21524.000000,21524.000000,2.152400e+04
mean,0.479744,63046.497661,43.292697,0.817274,0.972589,0.080886,1.653555e+05
std,0.755539,140827.311974,12.574476,0.548123,1.420341,0.272667,9.827105e+04
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.077151e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.448818e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,1.966055e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В dob_years в минимальном значении указоно 0, но нулей в возрасте быть не может, исправим это.

In [16]:
print('Нулей в столбце  dob_years до замены:', df['dob_years'][df['dob_years']==0].count())

Нулей в столбце  dob_years до замены: 101


Так как возраст клиента не требуется для ответа на поставленные вопросы, заполним нули простым способом - медианой по столбцу.

In [17]:
median_dob_years = df['dob_years'].median()
df['dob_years'] = df['dob_years'].replace(0, median_dob_years)
print('Нулей в столбце  dob_years после замены:', df['dob_years'][df['dob_years']==0].count())

Нулей в столбце  dob_years после замены: 0


### Вывод

Мы обработали пропущенные значения по ежемесячному доходу. Причинами таких пропусков может быть не предоставление клиентов вышеуказанной информации, но так же предполагаю, что банк не имеет информации о доходах, т.к. доходы клиент получает на счет в другом банке и к такой информации доступа нет (т.е 10% клиентов берут кредит в банке, в котором не обслуживаются). Так как это данные о доходе, то логично их было заполнить медианой, но чтобы значение было вернее медиану мы высчитали из сгруппированных данных по полу, образованию и виду деятельности.

Еще мы нашли некорректные данные в количестве детей, а эти данные очень нужны нам для нашего исследования. Мы предположили, что данные ошибочные и заменили неверные на предполагаемые.

После финальной провреки мы обнаружили нули в возрасте клиента и так как возраст не влияет на наше исследование заменили данные для красоты простым методом - заменили пропуски на медиану по столбцу.

### Замена типа данных

Для удобства переведем данные о доходе в цельночисленные методом astype().

In [18]:
df['total_income'] = df['total_income'].astype(int)

### Вывод

Одним из вопросов в нашем анализе будет: есть ли зависимость между уровнем дохода и возвратом кредита в срок. Для удобства дальнейше работы мы поменяли данные в столбце с доходом на цельночисленные.

### Обработка дубликатов

Предположим, что во фрейме есть полнностью дублированные строки. Применим к фрейму метод drop_duplicates и обновим индексы.

Но для начала уберем дубликаты по регистру в столбце education. Приведем столбцы с образованием к нормальному виду.

In [19]:
df['education'] = df['education'].str.lower()
print(' Проверка после применения метода str.lower:', df['education'].unique())

 Проверка после применения метода str.lower: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Тот же метод для верности применим и к столбцу с семейныым положением.

In [20]:
df['family_status'] = df['family_status'].str.lower()

Выведем фрейм с полностью дублированными строками.

In [21]:
df[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,124127,покупка жилья для семьи
3290,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,112790,сыграть свадьбу
4182,1,NaN,34,высшее,0,гражданский брак,1,F,сотрудник,0,153691,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,112790,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,112790,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,среднее,1,женат / замужем,0,F,пенсионер,0,112790,дополнительное образование
21032,0,NaN,60,среднее,1,женат / замужем,0,F,пенсионер,0,112790,заняться образованием
21132,0,NaN,47,среднее,1,женат / замужем,0,F,сотрудник,0,124127,ремонт жилью
21281,1,NaN,30,высшее,0,женат / замужем,0,F,сотрудник,0,153691,покупка коммерческой недвижимости


Полные дубликаты есть только в тех строках, которые не имели данные о стаже работы, а следовательно и данные о доходе. Удалим полностью дублированные строки.

In [22]:
shape_table = df.shape
df_duplicates = df.duplicated().sum()
print('Текущий размер', shape_table)
#print('Количество найденных дубликатов', df_duplicates)

Текущий размер (21524, 12)


In [23]:
shape_table = df.shape
df = df.drop_duplicates().reset_index(drop = True)
shape_table_update = df.shape
df_duplicates_after = df.duplicated().sum()

In [24]:
if shape_table_update == shape_table:
    print('Результат не изменился, текущий размер', shape_table_update)
    print('Дубликатов не обнаружено')
else:
    print('Таблица уменьшилась, текущий размер', shape_table_update)
    print('Найдено и удалено дубликатов:', df_duplicates)
        

Таблица уменьшилась, текущий размер (21452, 12)
Найдено и удалено дубликатов: 72


### Вывод

В таблице присутсвовали полностью дублированные строки. Всего найдено и удалено 72 дублированные строки. Так как по столбцам присутсвует много дубликатов, мы применили метод drop_duplicates потому, что он применятся сразу ко всему фрейму. Как оказалось простой метод оказался пригодным для данной таблицы. 

Причиной возникновения дубликатов может являться то, что клиент направил две заявки на кредит (нажал в банковском приложении два раза на кнопку "подать заявку на кредит").

### Лемматизация

Попробуем рассмотреть, какие данные содержатьяся в целях кредита.

In [25]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                605
на покупку своего автомобиля              505
заняться высшим образованием      

Мы теперь примерно понимаем какие данные присутсвуют в этом столбце. Но это неудобно.

Ипортируем библиотеку pymystem3 и лемматизируем строку text в которой мы сохранили уникальные значения по столбцу "цель кредита". После посчитаем число упоминаний лемматизированных слов при помощи контейнера Counter из модуля collections.

In [26]:
#from pymystem3 import Mystem
m = Mystem()
text = ' '.join(df['purpose'].unique())
lemmas = m.lemmatize(text)
#from collections import Counter
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


### Вывод

Мы лемматирзировали данные в столбце с целью кредита. Это будет использовано нами для категоризации данных. Но уже сейчас видно цели кредита это покупка недвижимости или автомобиля, кому-то нужны деньги на свадьбу или образование, а кто-то планирует сделать ремонт. 

### Категоризация данных

На основании списка наших лемм мы можем категоризовать данные, разделим данные по основным каттегориям - покупка недвижимости, ремонт, покупка автомобиля и проведение свадьбы.

Для того, чтобы категоризировать данные о цели кредита напишем функцию purpose_group, которая будет принимать значение lemmas и возвращать значения, согласно наших условий.

In [27]:
def purpose_group(lemmas):
    if 'ремонт' and 'жилье' in lemmas:
        return 'ремонт'
    if 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'жилье' in lemmas:
        return 'недвижимость'
    if 'операция' in lemmas:
        return 'недвижимость'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'строительство' in lemmas:
        return 'недвижимость'
    else:
        return 'другое'

Добавим новый стобец с леммами по каждой строке.

In [28]:
df['lemmas_column'] = df['purpose'].apply(m.lemmatize)

После добавим столбец purpose_category, применим написанную нами функцию к значениям по столбцу lemmas_column.

In [29]:
df['purpose_category'] = df['lemmas_column'].apply(purpose_group)

Теперь посчитаем значения по нашим категориям.

In [30]:
df['purpose_category'].value_counts()

недвижимость    6350
ремонт          4460
автомобиль      4306
образование     4013
свадьба         2323
Name: purpose_category, dtype: int64

Напишем функцию для категоризации клиентов по количеству детей.

In [31]:
def children_value(children):
    if children == 0:
        return 'нет детей'
    if children == 1:
        return 'один ребенок'
    if children == 2:
        return 'двое детей'
    if children > 2:
        return 'многодетный'

Создадим новый столбец children_type и применим метод apply для применения этой функции к значениям по столбцу children.

In [32]:
df['children_group'] = df['children'].apply(children_value)
df['children_group'].value_counts()

нет детей       14089
один ребенок     4855
двое детей       2128
многодетный       380
Name: children_group, dtype: int64

Для ответа на поставленные вопросы нам понадобиться  выделить группы и по уровню дохода. Построим сводную таблицу и узнаем данные о доходах. Чтобы потом выделить их в категории.

In [33]:
df.pivot_table(index = ['gender'], values = 'total_income', aggfunc = {'min', 'mean', 'median', 'max'})

,max,mean,median,min
gender,,,,
F,1715018.0,152249.644465,132635.0,20667.0
M,2265604.0,191215.009893,167084.0,21205.0
XNA,203905.0,203905.000000,203905.0,203905.0


Разброс от минимума до максимума очень большой. Поэтому применим метод qcut, чтобы разделить значения столбца на одинаковые по количеству группы.

Также мы одноружили значение XNA, но так как мы не планируем анализировать возрат кредита по половому признаку, то просто предположим, что это мужчина и заменим значение в gender  на M.

In [34]:
df['gender'] = df['gender'].replace('XNA', 'M')

Теперь выделим  равноразмерные категории методом qcut в новый столбец income_group.
Сделаем 6 категорий:
* очень низкий;
* низкий;
* средний;
* выше среднего;
* высокий;
* очень высокий.

In [35]:
df['income_group'] = pd.qcut(df['total_income'],6, labels=['очень низкий', 'низкий', 'средний', 'выше среднего', 'высокий', 'очень высокий'])
df['income_group'].value_counts()

средний          3656
очень высокий    3576
очень низкий     3576
высокий          3575
низкий           3575
выше среднего    3494
Name: income_group, dtype: int64

In [36]:
# reviewers code
df['total_income'].agg(['min','max'])

min      20667
max    2265604
Name: total_income, dtype: int64

### Вывод

Основной целью кредита является приобретение недвижимоси и ремнот недвижимости, в сумме эти категории составляют около 50% от всей выборки. Меньше всего клиенты берут кредиты на проведение свадьбы. 

Выделив клиентов по наличию детей в семье, мы видим, что почти 2/3 клиентов без детей.

Также мы выделили группы по уровню доходов методом qcut, таким образом группы получиллись равноразмерные.

### Шаг 3. Ответьте на вопросы

Перед тем как приступить к ответам на вопрос, посчитаем общую долю должников по всей выборке.

In [37]:
total_part_of_debetors = df['debt'].sum() / df['debt'].count()
total_part_of_debetors

0.08115793399216856

Около 8,12% процентов клиентов имели долги по кредитам.

Для дальнейшего удобства работы с данными создадим финальный фрейм в который занесем нужные значения для ответа на вопросы. Выведем на экран.

In [38]:
df_final = df[['children_group', 'family_status', 'family_status_id', 'income_group', 'purpose_category', 'debt']]
df_final.head()

,children_group,family_status,family_status_id,income_group,purpose_category,debt
0,один ребенок,женат / замужем,0,очень высокий,ремонт,0
1,один ребенок,женат / замужем,0,низкий,автомобиль,0
2,нет детей,женат / замужем,0,выше среднего,ремонт,0
3,многодетный,женат / замужем,0,очень высокий,образование,0
4,нет детей,гражданский брак,1,выше среднего,свадьба,0


- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
df_final_children = df_final.groupby('children_group').agg({'debt': ['count', 'sum']})
df_final_children['part'] = df_final_children['debt']['sum'] / df_final_children['debt']['count']
df_final_children.sort_values('part')

debt            part
                count   sum          
children_group                       
нет детей       14089  1063  0.075449
многодетный       380    31  0.081579
один ребенок     4855   445  0.091658
двое детей       2128   202  0.094925

### Вывод

Да, есть зависимость между наличием детей и возвратом кредита в срок. Меньше всего проблем с возвратом долга у клиентов без детей - только 7,55% из этой группы имели долги по кредиту - нет детей, нет долгов, можно платить за себя любимого=). Интересный факт, что если клиент является многодетным (3 и более детей), то возвращает кредит он лучше, чем клиенты с одним и двумя детьми - у мнгодетных 8,15% невозвратов, что очень близко к среднему по всей выборке. Видимо, у многодетных с ростом числа детей растет жизненная ответственность - нельзя рисковать и не отдавать кредит, когда под твоим "крылом" куча детей. Разница в процентах между клиентами без детей и клиентами с двумя детьми - 2%, разница существенная и поэтому мы смело можем утверждать, что зависимость есть. У клиентов с одним ребенком - 9,16% невозвратов, у клиентов с двумя - 9,49%, это близкие по показателям группы, но при наличие второго ребенка ситуация с возвратом долго усложняется.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
df_final_fam_status = df_final.groupby('family_status').agg({'debt': ['count', 'sum']})
df_final_fam_status = df_final.groupby('family_status').agg({'debt': ['count', 'sum']})
df_final_fam_status['part'] = df_final_fam_status['debt']['sum'] / df_final_fam_status['debt']['count']
df_final_fam_status.sort_values(by=['part'], ascending=False)

debt           part
                       count  sum          
family_status                              
не женат / не замужем   2810  274  0.097509
гражданский брак        4150  388  0.093494
женат / замужем        12338  931  0.075458
в разводе               1195   85  0.071130
вдовец / вдова           959   63  0.065693

In [41]:
df_final.pivot_table(index = ['children_group'], columns='family_status', values = 'debt', aggfunc = {'count'})

count                                                  \
family_status  в разводе вдовец / вдова гражданский брак женат / замужем   
children_group                                                             
двое детей            83             24              355            1582   
многодетный           12              7               66             285   
нет детей            784            847             2729            7467   
один ребенок         316             81             1000            3004   

                                      
family_status  не женат / не замужем  
children_group                        
двое детей                        84  
многодетный                       10  
нет детей                       2262  
один ребенок                     454

### Вывод

Мы знаем, что клиенты без детей имели меньше "просрочек", чем остальные, но интересно, что если рассматривать зависимость между семейным положением и возвратом кредита в срок, то тут лидером-неплательщиком будут свободные (не женат/не замужем) люди. Это значит, что если у клиента есть дети, это совсем не означает, что он женат/замужем.

Самые дисциплинированнеы вдовы/вдовцы - 6,56% невозвратов - по сводной таблице, видно, что большой процент таких клиентов не имеют детей, соответсвенно, отсутвие детей и потеря любимого человека делает таких людей внимательными к своим долгам (у них нет моментов, когда они тратят средства на близких людей и с возвратом долго проблем не возникает).

Клиенты в разводе (7,11%) и клиенты в официальных отношениях (7,54%) возвращают долги лучше, чем в среднем по всей выборке (8,12%).

Клиенты, которые не оформляли свои отношения или являются не женатыми/не замужними платить не любят 9,34% и 9,75%, соответсвенно. Это большое отклонение от среднего. Нет ответсвенности с оформлением отношений, значит и нет отвественности по срокам погошения долга в срок.

Те клиенты, которые сейчас в официальных отношения(женат/замужем) или когда-то были в них (вдовы/вдовцы и разведенные) имеют меньший процент невозвратов, чем те кто официальных оформлений не любят.

Зависиость есть - не оформляешь официально отношения, значит и платишь по кредиту хуже.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
df_final_income = df_final.groupby('income_group').agg({'debt': ['count', 'sum']})
df_final_income = df_final.groupby('income_group').agg({'debt': ['count', 'sum']})
df_final_income['part'] = df_final_income['debt']['sum'] / df_final_income['debt']['count']
df_final_income.sort_values(by=['part'], ascending=False)

debt           part
              count  sum          
income_group                      
выше среднего  3494  322  0.092158
средний        3656  316  0.086433
низкий         3575  294  0.082238
очень низкий   3576  285  0.079698
высокий        3575  278  0.077762
очень высокий  3576  246  0.068792

### Вывод

Мы выделили 6 групп по уровню дохода.

И, конечно, взаимосвязь есть. 
Клиенты с высоким и очень высоким доходом ведут себя добросовествно - 7,78% и 6,88% невозвратов в срок, здесь видно, что с ростом дохода, когда доход уже высокий, процент невозврата снижается.

Клиенты с очень низким доходом возвращают кредит лучше, чем в среднем по выборке - 7,97%. У таких клиентов и так проблем хватает и они не жалают сталкиваться с проблемами от невозврата кредита в срок (общение с приставами им еще не хватало или, не дай бог, исполнительного производства). Вот почему много кредитных рекламных компаний ориентировано на покупку чего-то не очень дорогого, как раз подходящего для клиентов с очень низким доходом. Подсаживаем Россию на кредитную иглу, ведь бедные хорошо платят по счетам.

Клиенты с низким доходом, тоже недалеко от среднего - 8,22%, но уже выше. И тут прослеживается взаимосвязь - от очень низкого дохода до дохода выше среднего процент невозврата равномерно растет (8,2% - 8,6% - 9,2%), значит у клиентов с ростом дохода появляются потребности в дорогом жилье/автомобиле, но они не рассчитывают свои силы и не могут платить в срок за свой новенький мерседес (хороший понт дороже денег). 

И когда доход уже становится высоким, то у клиента встают мозги на место и он платит по кредиту хорошо.



- Как разные цели кредита влияют на его возврат в срок?

In [43]:
df_final_purpose = df_final.groupby('purpose_category').agg({'debt': ['count', 'sum']})
df_final_purpose = df_final.groupby('purpose_category').agg({'debt': ['count', 'sum']})
df_final_purpose['part'] = df_final_purpose['debt']['sum'] / df_final_purpose['debt']['count']
df_final_purpose.sort_values(by=['part'], ascending=False)

debt           part
                 count  sum          
purpose_category                     
автомобиль        4306  403  0.093590
образование       4013  370  0.092200
свадьба           2323  186  0.080069
недвижимость      6350  474  0.074646
ремонт            4460  308  0.069058

### Вывод

Автомобилисты любят кататься, а за машину платить не очень - 9,4% из этой выборки имели долги по кредиту. Может быть это связано с тем, что сейчас часто покупают машины молодые люди - ответсвенности мало (нет ни детей ни семьи), недвижимость и свадьба пока не нужны, а машина самое то.

Образование - 9,2% - сокрее всего на образование берут кредит молодые люди (студенты) и у них небольшой доход, что приводит их к нестабильному графику выплат по кредиту.

Дальше в порядке жизненного цикла человека рассположились - свадьба, недвижимость и ремонт. И чем дальше идет жизненный цикл, тем меньше сбоев по оплате долгов. Человек взрослеет, играет свадьбу (8% неплательщиков), потом покупает квартиру (7,46% неплательщиков) и потом делает ремонт (6,9% неплательщиков).



### Шаг 4. Общий вывод

##### Общая информация
Целью нашего проекта было произвести анализ данных о клиентах банка для того, чтобы понять влияет ли семейное положение и количество детей на факт погашения кредита в срок, а после применить выводы анализа для построения кредитного скорринга (посмотрим, как распределяться scores среди участников).

На входе мы имеем информацию о клиентах банка и об их кредитной истории:
* children — количество детей в семье;

* days_employed — общий трудовой стаж в днях;

* dob_years — возраст клиента в годах;

* education — уровень образования клиента;

* education_id — идентификатор уровня образования;

* family_status — семейное положение;

* family_status_id — идентификатор семейного положения;

* gender — пол клиента;

* income_type — тип занятости;

* debt — имел ли задолженность по возврату кредитов;

* total_income — ежемесячный доход;

* purpose — цель получения кредита.


На стадии ознакомления с данными было выявлено, что данные о трудовом стаже клиента в днях (days_employed) - ошибочны, в столбце есть отрицательные значения, а также значения с очень большим количеством дней. Для поставленной задачи эти данные не требуеются, в данном анализе они использоваться не будут, но в дальнейшем требуется проследить есть ли проблемы при выгрузке этой информации из базы данных (исключить техническую проблему или понять каким образом закодированы эти значения).

##### Подготовка данных. Поиск и устранение ошибок в данных.
Были обнаружены пропуски в столбцах days_employed и total_income, одновременно пропуски в 2174 строках. Предположение о том, что это строки безработных или не имеющих стажа клиентов, было проверено и опровергнуто, т.к. вид деятельности по пропущенным значениям - пенсионер, госслужащий, компаньон, сотрудник, предприниматель. Скорее всего эти данные не предоставил сам клиент или банк не имеет сведений о доходах данных клиентов, т.к. клиент не обслуживается в этом банке (т.е 10% клиентов берут кредит в банке, в котором не обслуживаются). По сгруппированным по полу, типу дохода и уровню образованию данным мы рассчитали медианные значения и заполнили пропуски о ежемесячном доходе. На одного клиента мы не смогли рассчитать медианное значение уровня дохода и было принято решение исключить строку из общей выборки - на общий итог информация об одном клиенте не повлияет.

Ошибочные данные были также найдены в количестве детей клиента. Этот столбец мы не могли оставить без исправлений - данные очень сильно нужны нам для проведения анализа. Были исправлены отрицательные значения на положительные. У 76 клиентов было по 20 детей, предположив, что эта ошибка при заполнении анкеты клиента ("залипание" клавиш на клавиатуре) и сравненив эту информацию с клиентами у которых чеверо детей (таких строк было 50), мы изменили 20 на 2 детей.

В столбце с возврастом клиента были нулевые значения - это ошибка, но на решение поставленной задачи эти данные не влияют. Мы решили без группировки заполнить нули медианами по столбцу, всего таких нулей было 101 (около 0.5% от всех клиентов).

В столбце с полем мы обноружили значение XNA у одного клиента, но так как мы не планируем анализировать возврат кредита по половому признаку, то исходя из уровня дохода (203905) предположим, что это мужчина и заменим значение в gender на M.

Далее мы приступили к поиску и удалению дубликатов. В первую очередь привели в одному регистру значения по полю образование. После этого применили метод по удалению полностью дублированных строк, всего удалено 72 дублированные строки. Причиной возникновения дубликатов может являться то, что клиент направлял несколько заявок на кредит (нажал в банковском приложении два раза на кнопку "подать заявку на кредит"), ведь как мы предполагали данных о количестве детей - клавиатура могла "залипать".

Для удобства работы мы изменили тип данных в столбце с доходом на цельночисленный.


##### Лемматизация целей кредита.
Просмотрев уникальные значения целей кредита, мы применили метод лемматизации по этому полю и выявили основные цели кредита. На этапе лемматизации уже было видно на какие категории можно будет поделить поле с целью кредита - покупка недвижимости, покупка автомобиля, заё на проведение свадьбы, оплата образования, деньги на ремонт.

##### Выделение основных категорий замщиков для проведения анализа.
В первую очередь мы "разбили" клиентов по цели получения кредита:

* Покупка недвижимости - 6350 клиентов;
* Проведение ремонта - 4460 клиентов;
* Покупка автомобиля - 4306 клиентов;
* Деньги на образование - 4013 клиентов;
* Проведение свадьбы - 2323 клиентов.

Сделали группы по количеству детей:
* Без детей - 14089 клиентов;
* Один ребенок в семье - 2128 клиентов;
* Два ребенка в семье - 2128 клиентов;
* Многодетные - 380 клиентов.

По уровню дохода мы сделали равноразмерные группы, так как был очень большой разрыв от минимального дохода по столбцу до макисмального - в дальнейшем это позволит нам точнее рассмотреть зависимость между уровнем дохода и сроком возврата кредита в срок. Всего 6 групп:
* очень низкий - 3576 клиентов;
* низкий - 3575;
* средний - 3656;
* выше среднего - 3494;
* высокий - 3575;
* очень высокий - 3576.

### Результат исследования.
После предобработки данных мы получили **21452 клиентов**, которые подойдут нам для исследования зависимостей. Из них **8,12%** имели задолженности по выплате кредитов - это будет точкой для понимания последующего анализа.

###### Есть ли зависимость между наличием детей и возвратом кредита в срок?
Да, зависимость есть. Меньше всего проблем с возвратом долга у клиентов без детей - только **7,55%** из этой группы имели долги по кредиту - нет детей, нет долгов, можно платить за себя любимого=). Интересный факт, что если клиент является многодетным (3 и более детей), то возвращает кредит он лучше, чем клиенты с одним и двумя детьми - у мнгодетных **8,15%** невозвратов, что очень близко к среднему по всей выборке. Видимо, у многодетных с ростом числа детей растет жизненная ответственность - нельзя рисковать и не отдавать кредит, когда под твоим "крылом" куча детей. **Разница в процентах между клиентами без детей и клиентами с двумя детьми - 2%, разница существенная и поэтому мы смело можем утверждать, что зависимость есть.** У клиентов с одним ребенком - **9,16%** невозвратов, у клиентов с двумя - **9,49%**, это близкие по показателям группы, но при наличие второго ребенка ситуация с возвратом долго усложняется.

###### Есть ли зависимость между семейным положением и возвратом кредита в срок?
Мы знаем, что клиенты без детей имели меньше "просрочек", чем остальные, но интересно, что если рассматривать зависимость между семейным положением и возвратом кредита в срок, то тут лидером-неплательщиком будут свободные (не женат/не замужем) люди. Это значит, что если у клиента есть дети, это совсем не означает, что он женат/замужем.

Самые дисциплинированнеы вдовы/вдовцы - **6,56%** невозвратов - по сводной таблице, видно, что большой процент таких клиентов не имеют детей, соответсвенно, отсутвие детей и потеря любимого человека делает таких людей внимательными к своим долгам (у них нет моментов, когда они тратят средства на близких людей и с возвратом долго проблем не возникает).

Клиенты в разводе (**7,11%**) и клиенты в официальных отношениях (**7,54%**) возвращают долги лучше, чем в среднем по всей выборке (**8,12%**).

Клиенты, которые не оформляли свои отношения или являются не женатыми/не замужними платить не любят **9,34% и 9,75%**, соответсвенно. Это большое отклонение от среднего. Нет ответсвенности с оформлением отношений, значит и нет отвественности по срокам погошения долга в срок.

Те клиенты, которые сейчас в официальных отношения(женат/замужем) или когда-то были в них (вдовы/вдовцы и разведенные) имеют меньший процент невозвратов, чем те кто официальных оформлений не любят.

**Зависиость есть - не оформляешь официально отношения, значит и платишь по кредиту хуже.**

###### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Мы выделили 6 групп по уровню дохода.

И, конечно, взаимосвязь есть. Клиенты с высоким и очень высоким доходом ведут себя добросовествно - **7,78%** и **6,88%** невозвратов в срок, здесь видно, что с ростом дохода, когда доход уже высокий, процент невозврата снижается.

Клиенты с очень низким доходом возвращают кредит лучше, чем в среднем по выборке - **7,97%**. У таких клиентов и так проблем хватает и они не жалают сталкиваться с проблемами от невозврата кредита в срок (общение с приставами им еще не хватало или, не дай бог, исполнительного производства). Вот почему много кредитных рекламных компаний ориентировано на покупку чего-то не очень дорогого, как раз подходящего для клиентов с очень низким доходом. Клиенты с очень низким доходом хорошо платят по счетам.

Клиенты с низким доходом, тоже недалеко от среднего - **8,22%**, но уже выше. **И тут прослеживается взаимосвязь - от очень низкого дохода до дохода выше среднего процент невозврата равномерно растет (8,2% - 8,6% - 9,2%)**, значит у клиентов с ростом дохода появляются потребности в дорогом жилье/автомобиле, но они не рассчитывают свои силы и не могут платить в срок за свой новенький мерседес (хороший понт дороже денег).

И когда доход уже становится очень высоким, то у клиента нет проблем с оплатой долга, теперь кредит - это инструмент, а не потребность.

###### Как разные цели кредита влияют на его возврат в срок?
**Автомобилисты** любят кататься, а за машину платить не очень - **9,4%** из этой выборки имели долги по кредиту. Может быть это связано с тем, что сейчас часто покупают машины молодые люди - ответсвенности мало (нет ни детей ни семьи), недвижимость и свадьба пока не нужны, а машина самое то.

**Образование - 9,2%** - сокрее всего на образование берут кредит молодые люди (студенты) и у них небольшой доход, что приводит их к нестабильному графику выплат по кредиту.

Дальше в порядке жизненного цикла человека рассположились - свадьба, недвижимость и ремонт. И **чем дальше идет жизненный цикл, тем меньше сбоев по оплате долгов.** Человек взрослеет, играет **свадьбу** (**8%** неплательщиков), потом покупает **квартиру** (**7,46%** неплательщиков) и потом делает **ремонт** (**6,9%** неплательщиков).

### Итог
Мы получили необходимую информацию для построения модели кредитного скоринга, теперь банк может выделять клиентов, которым можно делать ставку по кредиту меньше, а с какими клиентами необходимо перестраховаться и "подкуртить" процент повыше. 

Применять полученные выводы можно для построения рекламной компании банка, если банк имеет хороший инструмент по контролю должников, то можно сделать упор на "рискованные" категории клиентов, но дать процент на кредит выше, но стоит помнить, что с увелечением прибыли риски в таких ситуация тоже растут.

Стоит также дать рекомендацию менеджерам по работе с клиентами банка:
Банку всегда стоит относится внимательно к сбору информации о клиентах и к заполнению анкет, а пока ошибки в данных встречаются и если их исключить, то исследования будут точнее, будет возможность проследить взаимосвязь между рабочем стажем и возвратом кредита в срок.

Спасибо за уделенное время!



### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.